In [1]:
import numpy as np 
import torch
from torch.utils.data import random_split, DataLoader
import torch.nn as nn
from pytorch_lightning import Trainer

# from torch.utils.tensorboard import SummaryWriter
# writer = SummaryWriter()

from dataset import MushroomBodyDataset 
from rnn import NeuralRNNModule, NeuralRNN

%load_ext autoreload
%aimport dataset, rnn
%autoreload 1

In [2]:
dataset = MushroomBodyDataset()

print(f'Datum shape for input and output is (timesteps, neurons): {dataset[0]["dan"].shape}')
# dataset.data[0]

Datum shape for input and output is (timesteps, neurons): torch.Size([3, 15])


In [3]:
num_samples = len(dataset.data)
train_percentage, val_percentage = 0.6, 0.2
test_percentage = 1 - train_percentage - val_percentage

lengths = [
    int(train_percentage * num_samples), 
    int(val_percentage * num_samples),
    int(test_percentage * num_samples),
]

train_dataset, val_dataset, test_dataset = random_split(dataset, lengths, generator=torch.Generator().manual_seed(42))

print(f'Train dataset size: {len(train_dataset)}')
print(f'Val dataset size: {len(val_dataset)}')
print(f'Test dataset size: {len(test_dataset)}')

Train dataset size: 5400
Val dataset size: 1800
Test dataset size: 1800


In [4]:
BATCH_SIZE = 8

train_dataloader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE,
    shuffle=False, 
    num_workers=0
)

val_dataloader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE,
    shuffle=False, 
    num_workers=0
)

# next(iter(train_dataloader))

In [5]:
# Single sample test
sample = train_dataset[0]['dan']
timesteps, input_features = sample.shape

sample = sample.unsqueeze(0)
print('Input shape', sample.shape)

print('Testing built in RNN')
rnn = nn.RNN(
    input_size=15, 
    hidden_size=15,
    num_layers=1,
    nonlinearity='tanh',
    bias=True,
    batch_first=True
)

output1, h_n1 = rnn(sample)

# (batch, seq_len, 1*hidden_size) = (1,3,15)
# tensor containing the output features (h_t) from the last layer of the RNN
print(output1.shape)

# num_layers * num_directions, batch, hidden_size = (1*1, 1, 15)
# Final hidden state : tensor containing the hidden state for t = seq_len.
print(h_n1.shape)

print('Testing my RNN')
my_rnn = NeuralRNN(
    input_size=15, 
    hidden_size=15
)

output2, h_n2 = rnn(sample)
print(output2.shape)
print(h_n2.shape)


Input shape torch.Size([1, 3, 15])
Testing built in RNN
torch.Size([1, 3, 15])
torch.Size([1, 1, 15])
Testing my RNN
torch.Size([1, 3, 15])
torch.Size([1, 1, 15])


In [6]:
output1

tensor([[[ 0.1817, -0.2094,  0.3375,  0.0886,  0.1764,  0.4102,  0.1239,
           0.0894, -0.2626,  0.0224,  0.0348, -0.0811,  0.2149,  0.3996,
          -0.0046],
         [ 0.3567, -0.3992,  0.2869, -0.1424,  0.1475,  0.5080, -0.1094,
           0.2515, -0.4954,  0.0657, -0.1514,  0.0449,  0.1507,  0.2951,
           0.1954],
         [ 0.3965, -0.3447,  0.2989, -0.0626,  0.1623,  0.4247,  0.0175,
           0.3142, -0.3226,  0.2091, -0.1140, -0.2171,  0.1495,  0.1899,
           0.0581]]], grad_fn=<TransposeBackward1>)

In [7]:
output2

tensor([[[ 0.1817, -0.2094,  0.3375,  0.0886,  0.1764,  0.4102,  0.1239,
           0.0894, -0.2626,  0.0224,  0.0348, -0.0811,  0.2149,  0.3996,
          -0.0046],
         [ 0.3567, -0.3992,  0.2869, -0.1424,  0.1475,  0.5080, -0.1094,
           0.2515, -0.4954,  0.0657, -0.1514,  0.0449,  0.1507,  0.2951,
           0.1954],
         [ 0.3965, -0.3447,  0.2989, -0.0626,  0.1623,  0.4247,  0.0175,
           0.3142, -0.3226,  0.2091, -0.1140, -0.2171,  0.1495,  0.1899,
           0.0581]]], grad_fn=<TransposeBackward1>)

In [8]:
batch = next(iter(train_dataloader))
output, h_n = my_rnn(batch['dan'])

print(output.shape)
# output

torch.Size([8, 3, 15])


# Lets Try Train

In [9]:
model = NeuralRNNModule(
    input_dim=15,
    hidden_dim=15, 
    lr=0.001
)

In [10]:
# Check that the forward pass does what I expect
output = model(batch['dan'])
output.shape

torch.Size([8, 3, 15])

In [11]:
from pytorch_lightning.loggers import TensorBoardLogger
logger = TensorBoardLogger("tb_logs", name="my_model")

In [12]:
trainer = Trainer(logger=logger)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [20]:
trainer.fit(model, train_dataloader, val_dataloader)


  | Name     | Type      | Params
---------------------------------------
0 | rnn      | NeuralRNN | 480   
1 | loss_fcn | MSELoss   | 0     
---------------------------------------
480       Trainable params
0         Non-trainable params
480       Total params
0.002     Total estimated model params size (MB)
Epoch 0:   1%|          | 10/900 [00:00<00:13, 65.54it/s, loss=0.0844, v_num=2]/anaconda3/envs/i2dl/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/anaconda3/envs/i2dl/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing